In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

## Tratamiento de datos de estaciones de medida meteorológicas de la AEMET

Los datos que se usarán en esta tarea proceden de la __[Agencia Estatal de Meteorología](http://www.aemet.es/es/portada)__.
Se han ido recopilando mediante sucesivas peticiones a la API REST de __[AEMET OpenData](http://www.aemet.es/es/datos_abiertos/AEMET_OpenData)__ y ahora están almacenados en el fichero `datos/opendata_AEMET.csv` que acompaña a este *Notebook*.

El fichero CSV contiene datos de medidas meteorológicas de estaciones en toda España con una frecuencia horaria, que abarcan desde el 11-05-2021 a las 18:00 hasta el 15-06-2021 a las 04:00.

<a id="sec_parametros_problema_ID"></a>
## Parámetros del problema

Vamos a plantearte una serie de cuestiones de extracción de información del dataset y algunas transformaciones. A continuación se definen algunos parámetros que te harán falta para resolver las cuestiones:

- fname: nombre del fichero de datos CSV de openData de AEMET
- fecha_ini: (str) fecha inicial
- fecha_fin: (str) fecha final 
- nombre_dominio: (str) nombre del dominio
- upper_right: (tupla: (lat, lon)) esquina superior derecha del bounding box del dominio
- lower_left: (tupla: (lat, lon)) esquina inferior izquierda del bounding box del dominio

In [ ]:
# Fichero CSV de datos openData de AEMET
fname = './datos/opendata_AEMET.csv'

# Fechas de interés (ambas se incluyen)
fecha_ini = '2021-05-15 00:00'
fecha_fin = '2021-06-12 18:00'

# Nombre y bounding box de la zona de interés
nombre_dominio = 'Baleares'
upper_right = (40.460101, 4.722377) # (latitud, longitud)
lower_left = (38.464513, 0.948573)  # (latitud, longitud)

## Leer y explorar el fichero
### Cuestiones básicas
* Tamaño del dataset (variables y observaciones)
* Obtén los nombre de las variables y su tipo de almacenamiento (entero, punto flotante, *object*, ...)
* ¿Hay filas duplicadas?
    * ¿Cuántas? 
    * Si las hay, eliminalas
* ¿Hay columnas con [varianza cero](https://www.r-bloggers.com/2014/03/near-zero-variance-predictors-should-we-remove-them/)?
    * ¿Cuántas?
    * Si las hay, eliminalas

In [ ]:
df_orig = pd.read_csv(fname)
df_orig.shape

Hay 641101 observaciones y 39 variables

In [ ]:
stds = df_orig.describe().std()
stds[stds < 10]

Ninguna variable tiene varianza cero.

## Escoger las variables que nos interesan
Los metadatos de opendata de AEMET están la siguiente URL:
https://opendata.aemet.es/opendata/sh/55c2971b

Hemos de quedarnos con las siguientes variables y eliminar las demás del dataset:
- Fecha y hora final del periodo de observación
- Identificador de las estaciones
- Ubicación de las estaciones (latitud, longitud y ubicación)
- Altitud de las estaciones
- Velocidad media del viento (sensor no ultrasónico)
- Dirección media del viento (sensor no ultrasónico)
- Temperatura instantánea del aire

Determina el tipo (continuo, discreto, categórico, ordinal, nominal, binario) de cada una de las variables.

In [ ]:
labels = [
	'fint',
	'idema',
	'lat',
	'lon',
	'ubi',
	'alt',
	'vv',
	'dv',
	'ta'
]
df = df_orig.filter(labels)
df.head()

## Conversión de tipos

En general es conveniente que las variables que representan fecha/hora sean de tipo *datetime*. En el dataset hay una variable que almacena la fecha/hora de cada observación.

* ¿Qué variable almacena la fecha/hora de la observación?
* ¿De qué tipo es?
* Conviértela a tipo *__[datetime](https://pandas.pydata.org/pandas-docs/stable/user_guide/timeseries.html#converting-to-timestamps)__* 

In [ ]:
df['fint'] = pd.to_datetime(df['fint'])
df.info()

## Selección de datos

El dataset contiene datos de todas las estaciones de AEMET en España a lo largo de varios meses de 2021. 

### Selección de las fechas

Nos interesa quedarnos con un subconjunto de esas medidas, seleccionando aquellas que estén comprendidas entre las fechas `fecha_ini` y `fecha_fin` (ver [Parámetros del problema](#sec_parametros_problema_ID))

In [ ]:
print('Before filtering: ', df_orig.shape)
df = df[(df['fint'] >= fecha_ini) & (df['fint'] <= fecha_fin)]
print('After filtering: ', df.shape)

### Selección de un área geográfica

Ahora debemos quedarnos únicamente con las estaciones que están en las islas Baleares. Para ello hemos determinado una zona geográfica en forma de rectángulo (bounding box) que está delimitado por su esquina inferior izquierda y su esquina superior derecha, conforme a los siguientes parámetros (tuplas):
* esquina inferior izquierda: `lower_left`
* esquina superior derecha: `upper_right`

En ambos casos el primer elemento de la tupla es la latitud y el segundo la longitud. Ver [Parámetros del problema](#sec_parametros_problema_ID).

<img src="datos/mapa_bbox.png" alt="Alt text" title="Title text" />

El rectángulo rojo representa el *bounding box*, que está definido por sus esquinas *lower left* y *upper right*.

## Conteo de NaNs

Identifica las variables que tienen valores faltantes y calcula cuántos valores le faltan a cada una.

## Identifica y elimina las estaciones que no tengan medidas de viento o de temperatura

El objetivo de este apartado es identificar y eliminar del dataset aquellas estaciones que no tengan ningún valor válido de viento (ya sea de magnitud o de dirección) o de temperatura.

Se pide:

* Identificación: obtener para cada estación, el nombre, el número de NaNs que tiene en cada una de las tres variables de interés (*vv*, *dv* y *ta*) y el porcentaje de NaNs sobre el número de medidas de esa estación.

* Eliminar del dataset aquellas estaciones que no tengan medidas de viento (magnitud o dirección) o de temperatura

Por ejemplo, dado el siguiente dataset, deben eliminarse las estaciones B (no tiene medidas de temperatura) y C (no tiene medidas de velocidad de viento); la información de la estación A ha de permanecer inalterada:

| idema | vv | dv  | ta |
|-------|----|-----|----|
| A     | 6  | 30  | NA |
| A     | 7  | 48  | 23 |
| A     | NA | NA  | 27 |
| B     | 4  | 120 | NA |
| B     | 6  | 96  | NA |
| C     | NA | 35  | 18 |
| C     | NA | 30  | 20 |
| C     | NA | 40  | 21 |



In [ ]:
df['idema'].unique().size

In [ ]:
grouped = df.filter(['idema', 'vv', 'dv', 'ta']).groupby('idema')
na_info  = (
	grouped
	.agg([lambda s:s.size-s.count(), lambda s: s.count() / s.size])
	.rename(columns={'<lambda_0>': 'missing', '<lambda_1>': 'ratio'})
)
na_info.head()

In [ ]:
has_data = grouped.count() != 0
have_data_labels = has_data[(has_data['vv'] & has_data['dv'] & has_data['ta'])].index
df = df[df['idema'].isin(have_data_labels)]
df.head()

## Imputación de valores faltantes
Aunque se hayan eliminado las estaciones que no tenían ningún valor de viento o de temperatura, aún quedan algunos NAs en las demás estaciones.

Imputa a los valores faltantes de viento (módulo y dirección) y temperatura la media de sus dos vecinos más cercanos **en el tiempo**, es decir, si falta un valor en la estación XYZ a las 10 horas, impútale la media de ese valor en esa misma estación a las 09 horas y a las 11 horas (si esos faltan, usa siempre el valor del más próximo que exista). 


Pista: __[df.ffill()](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.ffill.html?highlight=ffill#pandas.DataFrame.ffill)__ 
,
__[df.bfill()](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.bfill.html?highlight=bfill#pandas.DataFrame.bfill)__
y
__[df.fillna()](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.fillna.html#pandas.DataFrame.fillna)__

In [ ]:
# fill the NA values of the columns by interpolation with the previous and next values of the same location
group_by_station = df.sort_values(by=['fint']).groupby('idema')

transformed = group_by_station.transform(lambda s: s.fillna((s.bfill() + s.ffill()) / 2))

transformed.count(), df.count()

## Normalizar los nombres de las ubicaciones

No todos los nombres de las ubicaciones siguen el mismo formato, así que te vamos a pedir que los normalices de forma que:

- Todos los nombres estén en mayúsculas (p.e.: Sóller --> SÓLLER)
- No haya espacios al principio ni al final de los nombres (p.e.: 'BANYALBUFAR ' --> 'BANYALBUFAR')
- El carácter '/' debe ser sustituido por '-' (p.e.: 'MENORCA/AEROPUERTO' --> 'MENORCA-AEROPUERTO')
- No debe haber espacios repetidos (p.e.: 'CAMPOS-SALINES  LEVANT' --> 'CAMPOS-SALINES LEVANT')

Por supuesto, debes realizar esta normalización usando pandas. Los siguientes enlaces pueden ser útiles:

- [https://pandas.pydata.org/docs/user_guide/text.html](https://pandas.pydata.org/docs/user_guide/text.html)
- [https://pandas.pydata.org/docs/reference/series.html#api-series-str](https://pandas.pydata.org/docs/reference/series.html#api-series-str)

In [ ]:
def normalize_str(s):
  s = s.upper()
  s = s.strip()
  s = s.replace('/', '-')
  s = " ".join(s.split())
  return s

df['ubi'] = df['ubi'].map(normalize_str)

## Distribución de los datos de temperatura

Para cada una de las siguientes ubicaciones, representa un histograma de sus temperaturas; considera **solamente los 180 días más recientes** en los que hay datos de cada ubicación:

- Sóller
- Banyalbufar
- Sierra de Alfabia
- Capdepera-Faro
- Menorca/Aeropuerto

In [ ]:
stations = ('SÓLLER', 'BANYALBUFAR', 'SIERRA DE ALFABIA', 'CAPDEPERA-FARO', 'MENORCA/AEROPUERTO')
limited_df = df[df['ubi'].isin(stations)].head()

grouped_by_station = limited_df.groupby('ubi')


## Crear nuevas variables
El viento está expresado mediante su módulo (variable *vv* en m/s) y el ángulo desde el que sopla (variable *dir*) medido en grados sexagesimales referidos al norte (0 grados) y en el sentido horario, es decir:
* viento soplando desde el norte: 0 grados
* viento soplando desde el este: 90 grados
* viento soplando desde el sur: 180 grados
* viento soplando desde el oeste: 270 grados

En ocasiones conviene conocer la velocidad del viento expresada en sus componentes cartesianas $v_{x}, v_{y}$. 

Añade dos nuevas variables (columnas) al dataset (sean vx, vy), con las componentes de velocidad calculadas de la siguiente manera:

$$
v_{x} = m \ cos(\alpha)\\
v_{y} = m \ sin(\alpha)\\
\alpha = \frac{(270 - \theta) \ \pi}{180} \\
$$

donde $m$ es el módulo del viento, $\theta$ es el ángulo medido respecto al norte; $\alpha$ está expresado en radianes.


In [ ]:
alpha = (270 - df['dv']) * np.pi / 180
vx = df['vv'] * np.cos(alpha)
vy = df['vv'] * np.sin(alpha)
df['vx'] = vx
df['vy'] = vy
df.head()

## Escritura de ficheros de resultados

Escribir un fichero CSV de cada estación por separado con la información correspondiente a cada una de ellas ordenada por fecha/hora.

Los ficheros han de tener cabecera, el separador de campos ha de ser la coma (",") y no debe tener índice en las filas.

Todos los ficheros generados han de guardarse en el directorio `datos`.

Los nombres de los ficheros han de tener el siguiente formato:
`ID_ESTACION_YYYYMMDD.CSV`, donde:

* `ID_ESTACION` es el nombre de la estación
* `YYYY` es el año de la observación más reciente, con cuatro cifras
* `MM` es el número del mes de la observación más reciente, con dos cifras
* `DD` es el número del día de la observación más reciente, con dos cifras

**Pista**: ver __[sort_values()](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.sort_values.html?highlight=sort_values#pandas.DataFrame.sort_values)__
,
__[to_csv()](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.to_csv.html?highlight=to_csv#pandas.DataFrame.to_csv)__
y
__[strftime()](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.Timestamp.strftime.html?highlight=strftime)__

In [ ]:
data_dir = './datos/'


## Gráfica del módulo de viento

Crea una gráfica del módulo de viento de las estaciones *B013X* y *B051A* similar a la siguiente que aparece más abajo.

Para que las etiquetas del eje de abscisas (horizontal) se representen correctamente basta con hacer que el índice del dataframe contenga los valores de fecha/hora adecuadamente ordenados. A estos efectos, considera el uso de [df.set_index()](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.set_index.html) y de [df.sort_index()](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.sort_index.html).


<img src="./datos/grafica_vv.png" alt="grafica del modulo de viento en las estaciones B013X Y B051A" title="Title text" />